In [15]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from arch.unitroot import PhillipsPerron


**Load Data**

In [7]:
df = pd.read_csv("/Users/juanvalidmind/Dev/github/validmind/validmind-python/notebooks/datasets/lending_club_loan_rates.csv", sep='\t')

**ADF Metric**

In [12]:
adftest = adfuller(df['loan_rate_A'])
adftest

(-1.917289312690944,
 0.32397189281015515,
 1,
 135,
 {'1%': -3.479742586699182,
  '5%': -2.88319822181578,
  '10%': -2.578319684499314},
 -71.08908853191068)

**KPSS Metric**

In [14]:
kpsstest = kpss(df['loan_rate_A'])
kpsstest

/Users/juanvalidmind/Library/Caches/pypoetry/virtualenvs/validmind-eEL8LtKG-py3.10/lib/python3.10/site-packages/statsmodels/tsa/stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


(1.012356679488042,
 0.01,
 6,
 {'10%': 0.347, '5%': 0.463, '2.5%': 0.574, '1%': 0.739})

**Phillips-Perron (PP) Test**

In [16]:
pp = PhillipsPerron(df['loan_rate_A'])
pp

Test Statistic,-2.027
P-value,0.275
Lags,13


In [17]:
pp.lags

-2.027461766308415